In [ ]:
#source - https://youtu.be/e4PsaCF2odk?si=pxZEmuRaZBN0q2nf
pip install blankly

  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 4.8 MB/s eta 0:00:01
   ----------------------------------- ---- 1.6/1.8 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.1 MB/s eta 0:00:00
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-win_amd64.whl size=95034 sha256=212314e69df877364907097e6e8311acdaeec919bb4be47074299851305aadc8
  Stored in directory: c:\users\santi\appdata\local\pip

  error: subprocess-exited-with-error
  
  × Building wheel for newnewtulipy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [409 lines of output]
      C:\Users\santi\AppData\Local\Temp\pip-build-env-a3t4j_e1\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: GNU Lesser General Public License v3 (LGPLv3)
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
      c

In [5]:
!blankly init 

'blankly' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import blankly
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LSTM
import torch.optim as optim
from torch.autograd import Variable

ModuleNotFoundError: No module named 'blankly'